In [1]:
!pip install git+https://github.com/Evan-Kim2028/subgraph-query-portal.git

  Cloning https://github.com/Evan-Kim2028/subgraph-query-portal.git to /tmp/pip-req-build-6h3sasia
  Running command git clone --filter=blob:none --quiet https://github.com/Evan-Kim2028/subgraph-query-portal.git /tmp/pip-req-build-6h3sasia
  Resolved https://github.com/Evan-Kim2028/subgraph-query-portal.git to commit 2705650be81311b71ea6c34ae04875cfcd8fecc2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached subgrounds-1.4.0-py3-none-any.whl (66 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 696.4 kB/s eta 0:00:0000:0100:01
  Using cached pyarrow-11.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.9 MB)
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)
  Using cached pipe-2.0-py3-none-any.whl (8.8 kB)
  Using cached

In [14]:
import polars as pl
from queryportal.dex import Dex

from datetime import datetime, timedelta


import pandas as pd
# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

polars.config.Config

In [15]:
# define subgraph endpoint. This one is the Univ3 Ethereum endpoint maintained by Messari
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum'

# instantiate dex class. Dex stores dex-related query functions
my_dex = Dex(endpoint)

First query is to get PEPE pools on Univ3

In [24]:
pool_filter = {
    'pool_name_contains': 'Pepe'
}

In [34]:
pool_cols = [
    'pool_name', 
    'pool_id',
    'tokenIn_symbol', 
    'tokenOut_symbol', 
    'tokenIn_id',
    'tokenOut_id',
    'pool_totalValueLockedUSD'
    ]

# NOTE - Currently there's a bug where phantom columns "id" and "timestamp" appear even though the columns aren't selected. They refer to the swap hash ids and the timestamp that the swap occured. They can be ignored.
pool_df = my_dex.query_swaps(query_paths=pool_cols, filter_dict=pool_filter, query_size=10000, add_endpoint_col=False)

{'pool_': {'name_contains': 'Pepe'}}
tokenIn
tokenOut
pool
Shape: (10000, 9)
Column Names: ['id', 'timestamp', 'tokenIn_symbol', 'tokenIn_id', 'tokenOut_symbol', 'tokenOut_id', 'pool_name', 'pool_id', 'pool_totalValueLockedUSD']
Data Types: [Utf8, Utf8, Utf8, Utf8, Utf8, Utf8, Utf8, Utf8, Float64]
Data: 
shape: (5, 9)
┌───────────────────────────────────┬────────────┬────────────────┬───────────────────────────────────┬───┬───────────────────────────────────┬──────────────────────────────────┬───────────────────────────────────┬──────────────────────────┐
│ id                                ┆ timestamp  ┆ tokenIn_symbol ┆ tokenIn_id                        ┆ … ┆ tokenOut_id                       ┆ pool_name                        ┆ pool_id                           ┆ pool_totalValueLockedUSD │
│ ---                               ┆ ---        ┆ ---            ┆ ---                               ┆   ┆ ---                               ┆ ---                              ┆ ---              

In [35]:
unique_pool_df = pool_df[['pool_name', 'pool_id', 'pool_totalValueLockedUSD']].unique().sort(by='pool_totalValueLockedUSD', descending=True)

In [36]:
unique_pool_df = unique_pool_df.to_pandas()

In [37]:
unique_pool_df

,pool_name,pool_id,pool_totalValueLockedUSD
0,Uniswap V3 Pepe/Wrapped Ether 1%,0xf239009a101b6b930a527deaab6961b6e7dec8a6,1.831697e+06
1,Uniswap V3 Pepe/Wrapped Ether 1%,0xf239009a101b6b930a527deaab6961b6e7dec8a6,1.831631e+06
2,Uniswap V3 McPepes/Wrapped Ether 1%,0xdf19389d0b2bee33230d73f07c8e0e2c4e78fb59,2.746683e+05
3,Uniswap V3 Pepe/USD Coin 1%,0xcee31c846cbf003f4ceb5bbd234cba03c6e940c7,2.658515e+04
4,Uniswap V3 Wojak Coin/Pepe 1%,0x0b4cb9d4fb1cee079392bc7cf6417a669f4c2dbd,1.173221e+04
5,Uniswap V3 Pepe/McPepes 1%,0x5e63add89fad07394c866b5ec650d062be15ef73,2.869249e+02
6,Uniswap V3 PepeAI/Wrapped Ether 1%,0xe210cbb670aacb2d255b97f023fa401fcc053fec,1.450494e-14


In [ ]:
# it appears that the main pool liquidity for PEPE is this pool_id 0xf239009a101b6b930a527deaab6961b6e7dec8a6

In [44]:
swap_filter = {
    'timestamp_gte': int((datetime(2023, 4, 19) - timedelta(days=7)).timestamp()),
    'timestamp_lte': int(datetime(2023, 4, 19).timestamp()),
    'pool_id': '0xf239009a101b6b930a527deaab6961b6e7dec8a6'
}

swap_cols = [
    'account_id',
    'tokenIn_symbol', 
    'tokenOut_symbol', 
    'amountOutUSD', 
    'amountInUSD',
    'amountOut',
    'amountIn'
    ]

swap_df = my_dex.query_swaps(query_paths=swap_cols, filter_dict=swap_filter, query_size=100000, saved_file_name='pepe_swaps_4.12.23_to_4.19.23', add_endpoint_col=False)


{'pool_': {'id': '0xf239009a101b6b930a527deaab6961b6e7dec8a6'}}
account
tokenIn
tokenOut
Shape: (1000, 9)
Column Names: ['amountOutUSD', 'amountInUSD', 'amountOut', 'amountIn', 'id', 'timestamp', 'account_id', 'tokenIn_symbol', 'tokenOut_symbol']
Data Types: [Float64, Float64, Float64, Float64, Utf8, Utf8, Utf8, Utf8, Utf8]
Data: 
shape: (5, 9)
┌──────────────┬─────────────┬───────────┬───────────┬───┬────────────┬───────────────────────────────────┬────────────────┬─────────────────┐
│ amountOutUSD ┆ amountInUSD ┆ amountOut ┆ amountIn  ┆ … ┆ timestamp  ┆ account_id                        ┆ tokenIn_symbol ┆ tokenOut_symbol │
│ ---          ┆ ---         ┆ ---       ┆ ---       ┆   ┆ ---        ┆ ---                               ┆ ---            ┆ ---             │
│ f64          ┆ f64         ┆ f64       ┆ f64       ┆   ┆ str        ┆ str                               ┆ str            ┆ str             │
╞══════════════╪═════════════╪═══════════╪═══════════╪═══╪════════════╪══════════

In [45]:
swap_df

amountOutUSD,amountInUSD,amountOut,amountIn,id,timestamp,account_id,tokenIn_symbol,tokenOut_symbol
f64,f64,f64,f64,str,str,str,str,str
151.395821,151.724136,4.3048e26,7.6220e16,"""0x6bc0439ba2b0…","""1681918619""","""0x5039e0be707f…","""WETH""","""PEPE"""
99.335595,99.531048,2.8245e26,5.0000e16,"""0x5e4407ccead5…","""1681918619""","""0x431675f96ce0…","""WETH""","""PEPE"""
639.928935,651.571296,3.2147e17,1.8527e27,"""0x0b19bfae3874…","""1681918619""","""0x3559b592919d…","""PEPE""","""WETH"""
4976.552384,5059.898823,2.5000e18,1.4387e28,"""0xf85c7467bf2d…","""1681918607""","""0x4560a7d6422c…","""PEPE""","""WETH"""
70.710625,71.051933,2.0106e26,3.5693e16,"""0xcfb20f9b9a41…","""1681918607""","""0x5ad9aa1c7b1c…","""WETH""","""PEPE"""
118.594222,119.437257,3.3721e26,6.0000e16,"""0xcd548e754647…","""1681918607""","""0x452156c89a56…","""WETH""","""PEPE"""
5935.854864,6020.424472,2.9819e18,1.7118e28,"""0xc91b7b5eca0e…","""1681918607""","""0xdc089eb3bb39…","""PEPE""","""WETH"""
1001.514054,1017.12423,5.0312e17,2.8921e27,"""0x72f4fbbfe323…","""1681918607""","""0xa0e09f511d93…","""PEPE""","""WETH"""
995.310477,1013.155291,5.0000e17,2.8808e27,"""0x72e3c9e10447…","""1681918607""","""0x2e8f6f866259…","""PEPE""","""WETH"""
